In [1]:
using JuMP
using GLPK

In [2]:

A = [
    0 1 0 1 0 0 0 0 1 1;
    1 0 1 0 1 0 0 0 0 0;
    0 1 0 1 0 1 0 0 1 0;
    1 0 1 0 1 0 1 0 0 1;
    0 1 0 1 0 0 0 1 1 0;
    0 0 1 0 0 0 1 0 0 1;
    0 0 0 1 0 1 0 1 0 0;
    0 0 0 0 1 0 1 0 1 0;
    1 0 1 0 1 0 0 1 0 0;
    1 0 0 1 0 1 0 0 0 0;
]

c = [
    0  57  0  91  0  0  0  0  55  20;
    57  0  23  0  48  0  0  0  0  0;
    0  23  0  15  0  47  0  0  98  0;
    91  0  15  0  60  0  21  0  0  67;
    0  48  0  60  0  0  0  43  44  0;
    0  0  47  0  0  0  13  0  0  55;
    0  0  0  21  0  13  0  11  0  0;
    0  0  0  0  43  0  11  0  16  0;
    85  0  98  0  44  0  0  16  0  0;
    20  0  0  67  0  55  0  0  0  0;
]

t = [
    0  5  0  7  0  0  0  0  6  4;
    5  0  3  0  2  0  0  0  0  0;
    0  3  0  1  0  4  0  0  8  0;
    7  0  1  0  6  0  5  0  0  7;
    0  2  0  6  0  0  0  3  4  0;
    0  0  4  0  0  0  3  0  0  5;
    0  0  0  0  0  1  0  4  8  0;
    0  0  0  0  3  0  1  0  1  0;
    6  0  8  0  4  0  0  21  0  0;
    4  0  0  7  0  5  0  0  0  0;
]

n = size(A, 1)
start_vertex = 1
end_vertex = 8
T_max = 20

20

In [3]:
model = Model(GLPK.Optimizer)

@variable(model, 0 <= x[1:n, 1:n])

for i in 1:n
    if i == start_vertex
        cond = 1
    elseif i == end_vertex
        cond = -1
    else
        cond = 0
    end
    @constraint(model, sum((A[i, j] == 1) ? x[i, j] : 0 for j in 1:n) - sum((A[j, i] == 1) ? x[j, i] : 0 for j in 1:n) == cond)
end
# @constraint(model,time, sum(x[i, j] * t[i, j] for i in 1:n, j in 1:n) <= T_max)

@objective(model, Min, sum(x[i, j] * c[i, j] for i in 1:n, j in 1:n))

optimize!(model)

display(value.(x))
display(A)
if termination_status(model) == MOI.INFEASIBLE
    println("Nie ma rozwiazania problemu.")
else
    println("Najnizszy koszt to: ", objective_value(model))

    path = [start_vertex]
    current_vertex = start_vertex
    while current_vertex != end_vertex
        for j in 1:n
            if value(x[current_vertex, j]) ≈ 1
                push!(path, j)
                current_vertex = j
                break
            end
        end
    end

    println("Sciezka: ", join(path, " -> "))
end

10×10 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

10×10 Matrix{Int64}:
 0  1  0  1  0  0  0  0  1  1
 1  0  1  0  1  0  0  0  0  0
 0  1  0  1  0  1  0  0  1  0
 1  0  1  0  1  0  1  0  0  1
 0  1  0  1  0  0  0  1  1  0
 0  0  1  0  0  0  1  0  0  1
 0  0  0  1  0  1  0  1  0  0
 0  0  0  0  1  0  1  0  1  0
 1  0  1  0  1  0  0  1  0  0
 1  0  0  1  0  1  0  0  0  0

Najnizszy koszt to: 71.0
Sciezka: 1 -> 9 -> 8
